In [1]:
from torch import optim
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch
import random
import numpy as np
import torch.nn as nn
import albumentations as Albu
import pandas as pd
from torch.utils.data.sampler import RandomSampler
from warmup_scheduler import GradualWarmupScheduler
import os
from utils.dataset import PandasDataset, RGB2XYZTransform
from utils.metrics import model_checkpoint
from utils.train import train_model
from utils.models import EfficientNetApi, EfficientNetApiGem

In [2]:
seed = 42
shuffle = True
batch_size = 8
num_workers = 4
output_classes = 5
init_lr = 3e-4
warmup_factor = 10
warmup_epochs = 1
n_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
loss_function = nn.BCEWithLogitsLoss()

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

ROOT_DIR = '../..'

data_dir = '../../../dataset'
images_dir = os.path.join(data_dir, 'tiles')

Using device: cuda


In [3]:
load_model = efficientnet_b0(
     weights=EfficientNet_B0_Weights.DEFAULT
)
model = EfficientNetApiGem(model=load_model, output_dimensions=output_classes)
model = model.to(device)

In [4]:
print("Using device:", device)
loss_function = nn.BCEWithLogitsLoss()

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Using device: cuda


In [5]:
df_train_ = pd.read_csv(f"{ROOT_DIR}/data/train_5fold.csv")
df_train_.columns = df_train_.columns.str.strip()
train_indexes = np.where((df_train_['fold'] != 3))[0]
valid_indexes = np.where((df_train_['fold'] == 3))[0]
#
df_train = df_train_.loc[train_indexes]
df_val = df_train_.loc[valid_indexes]
df_test = pd.read_csv(f"{ROOT_DIR}/data/test.csv")

#### view data

In [6]:
(df_train.shape, df_val.shape, df_test.shape)

((7219, 5), (1805, 5), (1592, 4))

In [7]:
transforms = Albu.Compose([
    RGB2XYZTransform(),
    Albu.Transpose(p=0.5),
    Albu.VerticalFlip(p=0.5),
    Albu.HorizontalFlip(p=0.5),
])

valid_transforms = Albu.Compose([
    RGB2XYZTransform()
])

In [8]:
df_train.columns = df_train.columns.str.strip()

train_dataset = PandasDataset(images_dir, df_train, transforms=transforms)
valid_dataset = PandasDataset(images_dir, df_val, transforms=valid_transforms)
test_dataset = PandasDataset(images_dir, df_test, transforms=valid_transforms)

In [9]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, num_workers=num_workers, sampler=RandomSampler(train_dataset)
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=batch_size, num_workers=num_workers, sampler = RandomSampler(valid_dataset)
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, num_workers=num_workers, sampler = RandomSampler(test_dataset)
)

In [10]:
optimizer = optim.Adam(model.parameters(), lr = init_lr / warmup_factor)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs - warmup_epochs)
scheduler = GradualWarmupScheduler(optimizer, multiplier = warmup_factor, total_epoch = warmup_epochs, after_scheduler=scheduler_cosine)

In [11]:
train_model(
    model=model,
    epochs=n_epochs,
    optimizer=optimizer,
    scheduler=scheduler,
    train_dataloader=train_loader,
    valid_dataloader=valid_loader,
    checkpoint=model_checkpoint,
    device=device,
    loss_function=loss_function,
    path_to_save_metrics="logs/with-noise-xyz.txt",
    path_to_save_model="models/efficientnet-xyz.pth",
    patience=5,
)

Epoch 1/50



100%|██████████| 226/226 [02:36<00:00,  1.45it/s]


metrics {'val_loss': np.float32(0.33706138), 'val_acc': {'mean': np.float64(44.27096953392029), 'std': np.float64(1.171892221023603), 'ci_5': np.float64(42.43767261505127), 'ci_95': np.float64(46.20498716831207)}, 'val_kappa': {'mean': np.float64(0.7298986000985215), 'std': np.float64(0.01232557676247516), 'ci_5': np.float64(0.7082781031891044), 'ci_95': np.float64(0.7495382091581599)}, 'val_f1': {'mean': np.float64(0.3736350327432156), 'std': np.float64(0.011632533395105305), 'ci_5': np.float64(0.35374375283718107), 'ci_95': np.float64(0.39240901470184325)}, 'val_recall': {'mean': np.float64(0.3796866256594658), 'std': np.float64(0.01103874274868711), 'ci_5': np.float64(0.3609559327363968), 'ci_95': np.float64(0.3976366207003593)}, 'val_precision': {'mean': np.float64(0.4489036958217621), 'std': np.float64(0.017669090968495428), 'ci_5': np.float64(0.4194111436605453), 'ci_95': np.float64(0.4772007346153259)}}
Salvando o melhor modelo... 0.0 -> 0.7298986000985215
Epoch 2/50



100%|██████████| 226/226 [02:02<00:00,  1.85it/s]
/home/woshington/Projects/Doutorado/repo/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:1087: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  _warn_get_lr_called_within_step(self)


metrics {'val_loss': np.float32(0.28499892), 'val_acc': {'mean': np.float64(50.765650898218155), 'std': np.float64(1.1588111486812671), 'ci_5': np.float64(48.86426627635956), 'ci_95': np.float64(52.576178312301636)}, 'val_kappa': {'mean': np.float64(0.7782135102777165), 'std': np.float64(0.011035137480311962), 'ci_5': np.float64(0.7594752747647072), 'ci_95': np.float64(0.796247533091356)}, 'val_f1': {'mean': np.float64(0.4461217902004719), 'std': np.float64(0.012124918204641007), 'ci_5': np.float64(0.4263900279998779), 'ci_95': np.float64(0.46623875200748444)}, 'val_recall': {'mean': np.float64(0.44571535238623616), 'std': np.float64(0.011316592383648108), 'ci_5': np.float64(0.42788256853818896), 'ci_95': np.float64(0.4647971898317337)}, 'val_precision': {'mean': np.float64(0.5219291223883629), 'std': np.float64(0.013402630531479507), 'ci_5': np.float64(0.4997529312968254), 'ci_95': np.float64(0.5441346585750579)}}
Salvando o melhor modelo... 0.7298986000985215 -> 0.7782135102777165
Ep

...

metrics {'val_loss': np.float32(0.52590674), 'val_acc': {'mean': np.float64(61.49567866921425), 'std': np.float64(1.100800609622338), 'ci_5': np.float64(59.72298979759216), 'ci_95': np.float64(63.379502296447754)}, 'val_kappa': {'mean': np.float64(0.8168642755082525), 'std': np.float64(0.012116835793390264), 'ci_5': np.float64(0.7969778372547669), 'ci_95': np.float64(0.8360696993660603)}, 'val_f1': {'mean': np.float64(0.5610902594327927), 'std': np.float64(0.011854474650925638), 'ci_5': np.float64(0.5421401739120484), 'ci_95': np.float64(0.5805017203092575)}, 'val_recall': {'mean': np.float64(0.5626221778988838), 'std': np.float64(0.011555428195718037), 'ci_5': np.float64(0.5437200039625167), 'ci_95': np.float64(0.5821237325668335)}, 'val_precision': {'mean': np.float64(0.5804717517495155), 'std': np.float64(0.011819385579777594), 'ci_5': np.float64(0.5610912561416626), 'ci_95': np.float64(0.6004781275987625)}}
Epoch 13/50



100%|██████████| 226/226 [02:03<00:00,  1.83it/s]


metrics {'val_loss': np.float32(0.53392047), 'val_acc': {'mean': np.float64(63.56758998632431), 'std': np.float64(1.1127311551535035), 'ci_5': np.float64(61.7174506187439), 'ci_95': np.float64(65.42936563491821)}, 'val_kappa': {'mean': np.float64(0.8228741592215256), 'std': np.float64(0.012148501834921456), 'ci_5': np.float64(0.8028979042672931), 'ci_95': np.float64(0.8426812848781877)}, 'val_f1': {'mean': np.float64(0.5843468905091286), 'std': np.float64(0.011989093809077384), 'ci_5': np.float64(0.5653336018323898), 'ci_95': np.float64(0.6046482712030411)}, 'val_recall': {'mean': np.float64(0.5818142438530922), 'std': np.float64(0.01202504457419712), 'ci_5': np.float64(0.5626953780651093), 'ci_95': np.float64(0.601875239610672)}, 'val_precision': {'mean': np.float64(0.5918981665968895), 'std': np.float64(0.0120385566833749), 'ci_5': np.float64(0.5726122468709945), 'ci_95': np.float64(0.6120307743549347)}}
Epoch 14/50



100%|██████████| 226/226 [02:04<00:00,  1.82it/s]


metrics {'val_loss': np.float32(0.6020033), 'val_acc': {'mean': np.float64(63.535069370269774), 'std': np.float64(1.1231041116806173), 'ci_5': np.float64(61.66204810142517), 'ci_95': np.float64(65.37396311759949)}, 'val_kappa': {'mean': np.float64(0.8197788450224225), 'std': np.float64(0.0121766851231142), 'ci_5': np.float64(0.7999253816548043), 'ci_95': np.float64(0.8394051149141389)}, 'val_f1': {'mean': np.float64(0.5683405811190605), 'std': np.float64(0.01214805509823548), 'ci_5': np.float64(0.5485566645860672), 'ci_95': np.float64(0.5882485061883926)}, 'val_recall': {'mean': np.float64(0.565779797077179), 'std': np.float64(0.011643795231645845), 'ci_5': np.float64(0.547292348742485), 'ci_95': np.float64(0.5854935586452484)}, 'val_precision': {'mean': np.float64(0.5784571298956871), 'std': np.float64(0.012797417878024266), 'ci_5': np.float64(0.5565723180770874), 'ci_95': np.float64(0.599713659286499)}}

Early stopping at epoch 14. No improvement for 5 epochs.
Best epoch: 9 with kapp

# tests

In [13]:
from utils.metrics import evaluation, format_metrics
model.load_state_dict(
    torch.load(f"models/efficientnet-xyz.pth")
)
response = evaluation(model, test_loader, device)
result = format_metrics(response[0])
print(result)

100%|██████████| 199/199 [01:53<00:00,  1.76it/s]


VAL_ACC      Mean: 59.57 | Std: 1.18 | 95% CI: [57.60, 61.56]
VAL_KAPPA    Mean: 0.81 | Std: 0.01 | 95% CI: [0.79, 0.84]
VAL_F1       Mean: 0.54 | Std: 0.01 | 95% CI: [0.52, 0.56]
VAL_RECALL   Mean: 0.54 | Std: 0.01 | 95% CI: [0.52, 0.56]
VAL_PRECISION Mean: 0.55 | Std: 0.01 | 95% CI: [0.53, 0.57]
